# 1

发现乘积不能跨过0，所以以0分割成子任务。\
再根据负数的数量进行分割。

![](../imgs/152.svg)

In [ ]:
#include <iostream>
#include <vector>
#include <algorithm>
#include <numeric>
#include <limits>

using namespace std;

// @lc code=start
class Solution {
public:
    int maxProduct(vector<int>& nums) {
        // int max_val = *max_element(nums.begin(), nums.end());
        int max_val = INT32_MIN;    //0x80000000
        int tmp;
        vector<int>::iterator l = nums.begin();
        if(nums.size() == 1)    // 只有一个元素
            return nums[0];
        else if(nums.size() == 0)
            return max_val;
        for(auto r=nums.begin();; ++r){
            if(r == nums.end() || *r == 0){ // 判断顺序反的话，会发生越界访问
                if(l != r && (tmp = sub_max_product(nums, l, r)) > max_val) max_val = tmp;
                l = r+1;
            }

            if(r==nums.end()) break;
        }
        return max_val;
    }

    int sub_max_product(vector<int>& nums, vector<int>::iterator l, vector<int>::iterator r){
        vector<vector<int>::iterator> negative_iters;
        int sub_max_val;
        for(auto it=l; it!=r; ++it){
            if(*it < 0) negative_iters.push_back(it);
        }
        if(r - l == 1 && *l <= 0)   // 只有一个元素，且小于0
            return 0;
        if(negative_iters.size() % 2 == 0){
            sub_max_val = accumulate(l, r, 1, multiplies<int>());
        }else{
            sub_max_val = max(
                accumulate(l, negative_iters.back(), 1, multiplies<int>()), 
                accumulate(negative_iters.front()+1, r, 1, multiplies<int>())
            );
        }
        // cout << *l << ":" << *r << " " << sub_max_val << endl;
        return sub_max_val;
    }
};
// @lc code=end

int main(int argc, char const *argv[])
{
    vector<int> nums {
        -17, 0, 3, 5, -2, 10, 0, -10, 7, -6, 2, 0, -5, -10, 6, -3, 8, -10, 7, -4, 9, 6, 0, -2, -3, -4
        // 2, 3, -2, 4
        // -2, 0, -1
        // -2
    };
    cout << Solution().maxProduct(nums) << endl;
    return 0;
}

187/187 cases passed (8 ms)\
Your runtime beats 35.8 % of cpp submissions\
Your memory usage beats 36.99 % of cpp submissions (11.6 MB)

time: O(n)

space: O(n)

空间开销主要是在存储负数的iter，可以优化为O(1)，不过代码没这么简洁。

不如题解的动态规划简洁。

# 题解

动态规划

根据「53. 最大子序和」推出的状态转移方程：$fmax(i)=\mathop{max}_{i=1}^{n}\{f(i-1) \times a_i, a_i\}$，**这样是错误的**，因为不满足「最优子结构」（即当前位置的最优解未必是由前一个位置的最优解转移得到的。）。

**可以根据正负性进行分类讨论。**

考虑当前位置如果是一个负数的话，那么我们希望以它前一个位置结尾的某个段的积也是个负数，这样就可以负负得正，并且我们希望这个积尽可能「负得更多」，即尽可能小。如果当前位置是一个正数的话，我们更希望以它前一个位置结尾的某个段的积也是个正数，并且希望它尽可能地大。于是这里我们可以再维护一个 $fmin(i)$，它表示以第 i 个元素结尾的乘积最小子数组的乘积，那么我们可以得到这样的动态规划转移方程：
$$fmax(i)=\mathop{max}_{i=1}^{n}\{fmax(i-1) \times a_i, fmin(i-1) \times a_i, a_i\}$$
$$fmin(i)=\mathop{min}_{i=1}^{n}\{fmax(i-1) \times a_i, fmin(i-1) \times a_i, a_i\}$$

[ref](https://leetcode-cn.com/problems/maximum-product-subarray/solution/cheng-ji-zui-da-zi-shu-zu-by-leetcode-solution/)

In [ ]:
class Solution {
public:
    int maxProduct(vector<int>& nums) {
        vector <int> maxF(nums), minF(nums);
        for (int i = 1; i < nums.size(); ++i) {
            maxF[i] = max(maxF[i - 1] * nums[i], max(nums[i], minF[i - 1] * nums[i]));
            minF[i] = min(minF[i - 1] * nums[i], min(nums[i], maxF[i - 1] * nums[i]));
        }
        return *max_element(maxF.begin(), maxF.end());
    }
};

time: O(n)

space: O(n)

**考虑优化空间。**

由于第 i 个状态只和第 i-1 个状态相关，根据「滚动数组」思想，我们可以只用两个变量来维护 i-1 时刻的状态，一个维护fmax，一个维护fmin。

In [ ]:
class Solution {
public:
    int maxProduct(vector<int>& nums) {
        int maxF = nums[0], minF = nums[0], ans = nums[0];
        for (int i = 1; i < nums.size(); ++i) {
            int mx = maxF, mn = minF;
            maxF = max(mx * nums[i], max(nums[i], mn * nums[i]));
            minF = min(mn * nums[i], min(nums[i], mx * nums[i]));
            ans = max(maxF, ans);
        }
        return ans;
    }
};

time: O(n)

space: O(1)